In [1]:
from scipy import optimize as opt
import numpy as np
from scipy.optimize import minimize


# 目标函数
def objective(x):
	return x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + 8


# 约束条件
def constraint1(x):
	return x[0] ** 2 - x[1] + x[2] ** 2  # 不等约束


def constraint2(x):
	return -(x[0] + x[1] ** 2 + x[2] ** 2 - 20)  # 不等约束


def constraint3(x):
	return -x[0] - x[1] ** 2 + 2

def constraint4(x):
	return x[1] + 2 * x[2] ** 2 - 3  # 不等约束


# 边界约束
b = (0.0, None)
bnds = (b, b, b)

con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'ineq', 'fun': constraint2}
con3 = {'type': 'eq', 'fun': constraint3}
con4 = {'type': 'eq', 'fun': constraint4}
cons = ([con1, con2, con3, con4])  # 4个约束条件
x0 = np.array([0, 0, 0])
# 计算
solution = minimize(objective, x0, method='SLSQP',  bounds=bnds, constraints=cons)
x = solution.x

print('目标值: ' + str(objective(x)))
print('答案为')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))

目标值: 10.651091840572583
答案为
x1 = 0.5521673412903187
x2 = 1.2032591818518543


### 选址问题

某企业计划在美国的50个主要城市及其周边地区销售自己生产的产品，包括某种娱乐设施和某类服装。销售在网上进行。  
为了及时将网上的订货送达客户，需要在这50个城市中的一部分城市建立仓库，为此，企业调查了城市间的公路运输时间，数据在附件1中给出，其中的数字表示两个城市之间公路运输的需要天数。如从哥伦比亚到达拉斯需要3天，而到盐湖城则需要5天。企业还调查了这 50个主要城市及其周边地区的人口、该种类娱乐设施和服装的目前的消费量及税率，数据见附件2，其中娱乐设施的消费税率和州的一般消费品的消费稅率一致，服装的消费税率在有的州得到减免。  
例如，在 Phoenix 及其周边地区有 6.828,065人，该类服装年消费7107（千美元），该类娱乐设施年消费6308（千美元），消费品税率和服装消费税率都是5.6%。  
  
问题1：如果网上的订货需要在一天内送达，至少需要在多少个城市建仓库？在哪些城市建？  
  
问题2：如果网上购物需要按库所在地缴纳消费税，企业建仓库时要考虑为消费者减少消费税以保持企业产品的竞争力。请你建模分析：如果尽可能少建仓库，订货要保证一天内送达，而且尽可能为消费者减轻消费税负担，应该在哪几个城市建立仓库？并求出各仓库的规模比和它的供应范围。  
  
问题3：如果需要在美国建立一个娱乐设施工厂和一个服装厂为这些仓库供货，这两个厂应该分别建在哪里？

 因为题目的要求是一天内送达，所以我们系数矩阵保留所有两城市距离为一天的数据，其余数据都替换为0

In [ ]:
# 读取数据
data = pd.read_excel("数据位置")
# 将 Nan 替换为  0
data[1:50].fillna(0,inplace=True)
#删掉无用的列 具体按个人数据实例进行操作
del data['Unnamed: 0']
del data['到达']
data.drop([0],axis=0,inplace=True)
data[1:50].fillna(0,inplace=True)
#大于1的距离进行删除
reachable = data.values #将Excel表中数据存储到数组中
reachable[reachable > 1] = 0

### 覆盖问题

In [2]:
import pulp
#写问题
SetCoverLP = pulp.LpProblem("SetCover_problem_for_Warehouse", sense=pulp.LpMinimize)
#建立变量x
Zones = list(range(50))
x = pulp.LpVariable.dicts("Zone",Zones,cat="Binary") 
#目标函数    
SetCoverLP += pulp.lpSum(x[i] for i in range(50))
#限制条件
for j in range(50):
    SetCoverLP += (pulp.lpSum([x[i] * reachable[i][j] for i in range(50)]) >= 1)
    
SetCoverLP.solve()
 
print(SetCoverLP.name)
# 格式化输出
temple = "区域 %(zone)d 的决策是：%(status)s"  
# 获得最优解
y = np.array(range(50))
if pulp.LpStatus[SetCoverLP.status] == "Optimal":  
        for i in range(50):
            if x[i].varValue:#将建仓位置存储到数组中
                y[i] = 1
            else:
                y[i] = 0
            output = {'zone': i+1, 'status': '建仓' if x[i].varValue else '--'}
            print(temple % output)
        print("需要建立 {} 个仓库。".format(pulp.value(SetCoverLP.objective)))


NameError: name 'reachable' is not defined